In [1]:
import numpy as np
import pandas as pd
import collections as col
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.core.display import display, HTML
from plotly.offline import iplot, download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)


In [2]:
# загрузим датафрейм
df = pd.read_csv('stud_math.csv')
# посмотрим на название колонок,чтобы не было проблем при обращении
df.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'studytime, granular', 'higher', 'internet', 'romantic', 'famrel',
       'freetime', 'goout', 'health', 'absences', 'score'],
      dtype='object')

In [3]:
#имеется непонятная колонка 'studytime, granular',которой не было в описании датафрейма .Дропнем ее
df=df.drop(['studytime, granular'],axis=1)
df.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout',
       'health', 'absences', 'score'],
      dtype='object')

In [4]:
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,nursery,higher,internet,romantic,famrel,freetime,goout,health,absences,score
0,GP,F,18,U,NaN,A,4.0,4.0,at_home,teacher,...,yes,yes,NaN,no,4.0,3.0,4.0,3.0,6.0,30.0
1,GP,F,17,U,GT3,NaN,1.0,1.0,at_home,other,...,no,yes,yes,no,5.0,3.0,3.0,3.0,4.0,30.0
2,GP,F,15,U,LE3,T,1.0,1.0,at_home,other,...,yes,yes,yes,NaN,4.0,3.0,2.0,3.0,10.0,50.0
3,GP,F,15,U,GT3,T,4.0,2.0,health,NaN,...,yes,yes,yes,yes,3.0,2.0,2.0,5.0,2.0,75.0
4,GP,F,16,U,GT3,T,3.0,3.0,other,other,...,yes,yes,no,no,4.0,3.0,2.0,5.0,4.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2.0,2.0,services,services,...,yes,yes,no,no,5.0,5.0,4.0,4.0,212.0,45.0
391,MS,M,17,U,LE3,T,3.0,1.0,services,services,...,no,yes,yes,no,2.0,4.0,5.0,2.0,3.0,80.0
392,MS,M,21,R,GT3,T,1.0,1.0,other,other,...,no,NaN,no,no,5.0,5.0,3.0,3.0,3.0,35.0
393,MS,M,18,R,LE3,T,3.0,2.0,services,other,...,no,yes,yes,no,4.0,4.0,1.0,5.0,0.0,50.0


In [5]:
for col in df.columns:
    if df[col].dtype==type(object()):
        df[col]=df[col].apply(lambda x : x if isinstance(x,str) else '')
        df[col]=df[col].apply(lambda x: None if x.strip() == '' else x)
    if df[col].dtype!=type(object()):
        print(col)

      






age
Medu
Fedu
traveltime
studytime
failures
famrel
freetime
goout
health
absences
score


In [6]:
'''визуализируем по каждому столбцу гистограмму содержащихся типов'''
number=len(df.columns)
fig=make_subplots(rows=1,cols=number,column_titles=list(df.columns))
fig.update_layout(width=2500,height=800,showlegend=False)
objects=[go.Histogram(x=df[columns].apply(lambda x : str(type(x))),name=columns) 
         for columns in df.columns]
fig.add_traces(objects,
              rows=[1]*len(df.columns),
              cols=[i for i in range(1,number+1,1)])

In [7]:
'''строим распеределение признаков для столбцов с числовыми значениями'''
names=[columns for columns in df.columns if df[columns].dtype!=type(object())]
objects=[go.Box(y=df[columns],name=columns) for columns in df.columns if df[columns].dtype!=type(object())]
fig=make_subplots(rows=1,cols=len(objects),column_titles=names)
fig.update_layout(height=800,width=1600,showlegend=False)
fig.add_traces(objects,
               rows=[1]*len(objects),
               cols=[i for i in range(1,len(objects)+1,1)])











In [8]:
"теперь оценим графики и посмотрим что необходимо сделать"
#1.Возраст. Выброс 22 года, что вполне может быть. Возможно это афроамериканец, который все никак не может доучится
#2.No problems
#3.Образование отца:выброс 0-т.е без образования.Вполне может быть,возможно это отец того 22-летнего афроамериканца)
#4.Treveltime .Выброс вполне допустим,не все живут близко от школы
#5.Studytime.Выброс 4 -думаю допустимо. У каждого в классе была отличница ноулайфовая
#6.failures .Не совсем понятно что это за критерий

'теперь оценим графики и посмотрим что необходимо сделать'

In [9]:
'''построим распеределние нечисловых значений'''
objects = [go.Histogram(x=df[columns],   
                        name=columns) for columns in
           df.columns if df[columns].dtype==type(object)]
fig= make_subplots(rows=1, cols=len(objects),subplot_titles=[columns for columns in df.columns if df[columns].
                                                             dtype==type(object())])
fig.update_layout(height=800, width=1600, showlegend=False)
fig.add_traces(objects, 
               rows=[1]*len(objects),
               cols=[i for i in range(1, len(objects)+1, 1)])



     
        

In [10]:
df_digital=pd.DataFrame([df[col] for col in df.columns if df[col].dtype!=type(object())]).T
df_digital.corr()

        


,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,health,absences,score
age,1.000000,-0.161704,-0.138018,0.081508,-0.015556,0.228047,0.068899,0.029023,0.135107,-0.065946,0.093592,-0.155726
Medu,-0.161704,1.000000,0.275598,-0.173515,0.078589,-0.245463,-0.010050,0.025335,0.064164,-0.045051,-0.011944,0.213349
Fedu,-0.138018,0.275598,1.000000,0.030646,0.057199,-0.150821,0.040577,-0.066556,-0.023282,0.027251,0.004695,0.078573
traveltime,0.081508,-0.173515,0.030646,1.000000,-0.102206,0.064699,-0.016857,-0.021937,0.037636,-0.021531,-0.048215,-0.070559
studytime,-0.015556,0.078589,0.057199,-0.102206,1.000000,-0.178992,0.045894,-0.121969,-0.051280,-0.080016,-0.026544,0.115741
failures,0.228047,-0.245463,-0.150821,0.064699,-0.178992,1.000000,-0.022655,0.098382,0.120914,0.081284,0.057391,-0.344338
famrel,0.068899,-0.010050,0.040577,-0.016857,0.045894,-0.022655,1.000000,0.175743,0.081154,0.077474,-0.082609,0.057112
freetime,0.029023,0.025335,-0.066556,-0.021937,-0.121969,0.098382,0.175743,1.000000,0.296437,0.061575,0.009292,0.019026
goout,0.135107,0.064164,-0.023282,0.037636,-0.051280,0.120914,0.081154,0.296437,1.000000,-0.015608,-0.048738,-0.129685
health,-0.065946,-0.045051,0.027251,-0.021531,-0.080016,0.081284,0.077474,0.061575,-0.015608,1.000000,-0.016222,-0.070460


In [11]:
fig = px.scatter_matrix(df_digital)
fig.update_layout(width=1200,height=1200)


In [12]:
fig=px.imshow(df_digital.corr(),x=df_digital.corr().columns,y=df_digital.corr().index)
fig.update_layout(height=800,width=800)
fig.show()